<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini API の使い方

元ネタ：https://ai.google.dev/gemini-api/docs/get-started/tutorial?hl=ja&lang=python

## 準備

### Python SDKのインストール

[`google-generativeai`](https://pypi.org/project/google-generativeai/) パッケージをインストールする。

In [ ]:
!pip install -q -U google-generativeai

### パッケージのインポート



In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

### API keyのセットアップ

始める前に、API Key が必要です。以下のリンクから入手すること。

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


左のパネルの "🔑" をクリックし、`GEMINI_API_KEY`を作成し、入手した API Key を登録する。

In [ ]:
# Or use `os.getenv('GEMINI_API_KEY')` to fetch an environment variable.
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

## モデルのリスト

利用可能な言語モデルの一覧が `list_models` で得られる。（2025年2月現在、Genmini-2.0 が最新）

In [ ]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

注：APIは、１分間に15回までの利用が可能。

## テキストの生成

モデルの選択：

In [ ]:
model = genai.GenerativeModel("gemini-2.0-flash")

<a href="https://ai.google.dev/api/generate-content#v1beta.models.generateContent"><code>GenerativeModel.generate_content</code></a> メソッドに指示内容を入れる。`generate_content` メソッドに入力できるものはモデルによって異なる。gemini-2.0-flash では、テキスト、画像、音楽が入力可能。

In [ ]:
%%time
response = model.generate_content("防衛大学校の研究科について教えてください。")

結果は、`response.text` に入っている。to_markdown` function 関数で表示できる。

In [ ]:
to_markdown(response.text)

## 画像からのテキスト生成

Gemini 2.0 は、マルチモーダルによる問い合わせが可能。

画像を入手する。 --> image.jpg （左側のファイルアイコンを押すことで確認できる）

In [ ]:
!curl -o image.jpg https://www.news-postseven.com/uploads/2020/06/11/jiji_boueidai_boushinage.jpg

In [ ]:
import PIL.Image

img = PIL.Image.open("image.jpg")
img

`generate_contnt` 関数にプロンプトと画像のリストを入力すると答えが帰ってくる。

In [ ]:
response = model.generate_content(["この写真について説明してください。", img])

to_markdown(response.text)

## チャットによる会話

`ChatSession` により会話を行う。これは、`generate_content`と違い、履歴をプロンプトに含めて入力する必要はない。

チャットの初期化：

In [ ]:
chat = model.start_chat(history=[])
chat

`ChatSession.send_message` メソッドで返事が返ってくる。history に会話の履歴が格納される。

In [ ]:
response = chat.send_message("コンピュータとは何かを小学生にも分かるように一文で表してください。")
to_markdown(response.text)

In [ ]:
chat.history

前の答えを踏まえて会話を続けることができる。

In [ ]:
response = chat.send_message("了解。では、高校生向けに長文で説明してください。")

to_markdown(response.text)

In [ ]:
for message in chat.history:
    display(to_markdown(f"**{message.role}**: {message.parts[0].text}"))

### generate_content を使った会話

`genai.ChatSession` が自動でやっていたことを明示する必要がある。role とparts からなるリストを入力させる必要がある。--> `generate_content` はステートレス（以前の情報を覚えていない）

In [ ]:
messages = [
    {
        "role": "user",
        "parts": ["コンピュータとは何かを小学生にも分かるように一文で表してください。"],
    }
]
response = model.generate_content(messages)

to_markdown(response.text)

メッセージに以前の会話の内容を含める（append 関数の利用）。

In [ ]:
messages.append({"role": "model", "parts": [response.text]})

messages.append(
    {
        "role": "user",
        "parts": [
            "了解。では、高校生向けに長文で説明してください。",
        ],
    }
)

response = model.generate_content(messages)

to_markdown(response.text)

### 生成に関する設定

`generation_config` で、生成に関する設定事項を指定できる。

In [ ]:
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(
    "防衛大学校生と一般大の学生の恋愛小説を書いてください。",
    generation_config=genai.types.GenerationConfig(
        # Only one candidate for now.
        candidate_count=1,
        stop_sequences=["x"],
        max_output_tokens=1000,
        temperature=1.0,
    ),
)

In [ ]:
text = response.text

#if response.candidates[0].finish_reason.name == "MAX_TOKENS":
#    text += "..."
#
to_markdown(text)

## 音声からのテキスト生成
写真と同じように扱える。

In [ ]:
!curl -o sample.wav https://note.com/api/v2/attachments/download/cda970a6812a6284d83ea847abf8f08d

In [ ]:
from IPython.display import Audio
Audio("sample.wav")

### 音声ファイルの読み取り

In [ ]:
audio_file = genai.upload_file(path="sample.wav")

### 音声ファイルの要約

In [ ]:
response = model.generate_content(
    [
        "次のラジオの音声を日本語で要約してください。",
        audio_file
    ]
)
to_markdown(response.text)

### 音声ファイルから議事録

In [ ]:
response = model.generate_content(
    [
        "次の会議から議事録を作ってください。",
        audio_file
    ]
)
to_markdown(response.text)

### 音声ファイルから文字起こし

In [ ]:
response = model.generate_content(
    [
        "次の会話を文字起こししてください。形式は、「時刻：発言者：発言内容」としてください。",
        audio_file
    ]
)
#to_markdown(response.text)
print(response.text)

## 他のモデルの利用

In [ ]:
model = genai.GenerativeModel("gemini-2.0-flash-thinking-exp")

解いて欲しい問題を test.png という名前で保存し、左のパネルのファイルアイコンをクリックし、パネル内にアップロードしてください。

In [ ]:
import PIL.Image
img = PIL.Image.open("test.png")
img

In [ ]:
response = model.generate_content(
    [
        "この図に書かれている問題を解いてください。解答は日本語で書いてください。",
        img
    ]
)
to_markdown(response.text)